$r_{0}$ = 300 ;
$\beta$ = 2.1 ; 
$S_{0}$ = 35 ; 
$\sigma$ = $\sqrt{35}$

In [ ]:
from scipy.optimize import leastsq
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib.path as mpltPath
import math as math

In [ ]:
def getDistances(xpositions, ypositions, x0=0, y0=0, zenithRad=0, azimuthRad=0):

    dx = xpositions - x0    # detector wrt core positions
    dy = ypositions - y0
    
    groundDistances = np.sqrt(dx*dx+dy*dy)

    # Direction cosines in the plane xy
    dirx =  math.cos(azimuthRad)
    diry =  math.sin(azimuthRad)

    axisProjections = math.sin(zenithRad) * (dx*dirx+dy*diry)
    
    distances = np.sqrt( groundDistances**2 - axisProjections**2 )
    
    return distances

def simZenith(zenithMinDeg, zenithMaxDeg) :

    zenithMinRad = math.radians(zenithMinDeg)
    sin2MinZenith = math.sin(zenithMinRad)**2

    zenithMaxRad = math.radians(zenithMaxDeg)
    sin2MaxZenith = math.sin(zenithMaxRad)**2

    rng = np.random.default_rng()
    u = rng.random() 
    
    sin2Zenith = sin2MinZenith + u * (sin2MaxZenith-sin2MinZenith)
    sinZenith = math.sqrt(sin2Zenith) 

    zenith = math.asin(sinZenith)
    azimut = np.random.uniform(0,2*np.pi)
    
    return (zenith,azimut)

def exp(params, x, y):
    a, b = params[0], params[1]
    residual = y-(a*x**b)
    return residual

def in_hex(poli):
    path = mpltPath.Path(poli)
    hex = False
    random_point = (np.random.uniform(-433,433),np.random.uniform(-375,375))
    while hex != True:
        if path.contains_points([random_point]) == True:
            hex = True
        else:
            random_point = (np.random.uniform(-433,433),np.random.uniform(-375,375))
    return random_point
    
    
params = [1, 1]

def evento(poligono,pointsx,pointsy,sigma,evento,LDF,angulo):
    #Genero el punto random.
    random_point = in_hex(poligono)
    #calculo las distancias entre el evento y los 7 detectores.
    distancias = getDistances(np.asarray(pointsx),np.asarray(pointsy),random_point[0],random_point[1],simZenith(-angulo,angulo)[0],simZenith(-angulo,angulo)[1])
    #Genero la señal con una LDF
    y = 35*(distancias/300)**(-2.1)
    #Hago el ajuste
    y_random = np.random.normal(y,sigma,size = len(y))
    a1 = leastsq(func, params, (np.asarray(distancias), y_random))[0][1]
    a2 = leastsq(func, params, (np.asarray(distancias), y_random))[0][0]
    #Opcional visualizar el esquematico del evento
    if evento == True:
        plt.figure(figsize=(8,8))
        polygon = plt.Polygon(poligono,fill=None,edgecolor='r')
        plt.gca().add_patch(polygon)
        plt.xlim(-600,600)
        plt.ylim(-600,600)
        plt.scatter(centro[0],centro[1],s = 100,c ='black')
        plt.scatter(random_point[0],random_point[1],s = 50,c='violet')
        plt.scatter(pointsx,pointsy,s = 100,c = 'black')
        plt.grid(linestyle='--')
        plt.show()
    #Opcional visualizar el ajuste del evento
    if LDF == True:
        x =  np.linspace(100,800)
        y =  a1 + a2*x
        
        plt.figure(figsize=(7,7))
        plt.plot(x,y)
        plt.plot(distancias,y_random,'o',color = 'orange')
        plt.errorbar(distancias,y_random,yerr =sigma,ls = 'none',ecolor='black')
        plt.title('Ajuste del evento',size = 20)
        plt.grid(linestyle='--')
        plt.plot([],[],' ',label = f'Ordenada = {np.round(a1,3)}')
        plt.plot([],[],' ',label = f'Pendiente = {np.round(a2,3)}')
        plt.ylabel('y(x)',size = 16)
        plt.legend(fancybox=True, framealpha=1, shadow=True, borderpad=1)
        plt.xlabel('x',size = 16)
    return(distancias,y_random,np.round(a1,5),np.round(a2,5),np.mean(distancias))

poligono = [[-216.5,-375],[216.5,-375],[433,0],[216.5,375],[-216.5,375],[-433,0]]
rectangulo = [[-433,-375],[433,-433],[433,375],[-433,375]]
pointsx,pointsy = [-216.5,216.5,433,216.5,-216.5,-433.5,0] , [-375,-375,0,375,375,0,0]
centro = (0,0)